# Chat

`Chat` is an object for conversational LLM interactions that tracks history and token usage across single or multiple models.

In [1]:
from irouter import Chat

# To load OPENROUTER_API_KEY from .env file create a .env file at the root of the project with OPENROUTER_API_KEY=your_api_key
# Alternatively pass api_key=your_api_key to the Chat class
from dotenv import load_dotenv

load_dotenv()

True

In this notebook we will use free tiers for Moonshot AI's Kimi K2 and Google's Gemma 3N. 

An overview of all available models can be found by calling `get_all_models`:
```python
from irouter.base import get_all_models
model_slugs = get_all_models()
model_slugs
```

In [2]:
model_names = ["moonshotai/kimi-k2:free", "google/gemma-3n-e2b-it:free"]

# Test conversation messages
first_message = "Who played the guitar solo on Steely Dan's Kid Charlemagne?"
second_message = "What other songs did this guitarist play on with Steely Dan?"

# Single Model

The simplest way to use `Chat` is with a single LLM by providing a model slug. Unlike `Call`, `Chat` maintains conversation history and tracks token usage.

In this example we initialize a `Chat` object with the free tier of Moonshot AI's Kimi-K2 LLM.

To set the API key you can either set an environment variable for `OPENROUTER_API_KEY` to your project or pass `api_key` when initializing `Chat`.

In [3]:
c = Chat(model_names[0], system="You are the best assistant in the world.")
# or
# c = Chat(model_names[0], api_key="your_api_key")

At the start the `history` will only contain the system message.

In [4]:
c.history

[{'role': 'system', 'content': 'You are the best assistant in the world.'}]

`Chat` will also tracks the token usage.

In [5]:
c.usage

{'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

In [6]:
c("Hello who are you?")

"Hello! I'm Kimi, your intelligent assistant and good friend. I'm here to help you with any questions or tasks you have. How can I assist you today?"

In [7]:
c.history

[{'role': 'system', 'content': 'You are the best assistant in the world.'},
 {'role': 'user', 'content': 'Hello who are you?'},
 {'role': 'assistant',
  'content': "Hello! I'm Kimi, your intelligent assistant and good friend. I'm here to help you with any questions or tasks you have. How can I assist you today?"}]

In [8]:
c.usage

{'prompt_tokens': 25, 'completion_tokens': 34, 'total_tokens': 59}